In [3]:
# docs: https://wbdata.readthedocs.io/en/stable/
# data: https://data.worldbank.org/indicator
import wbdata as wb

# docs: https://geocoder.readthedocs.io/providers/GeoNames.html
# data: https://www.geonames.org/countries/
#import geocoder

#import mpu
import re
import pandas as pd
from datetime import date

import sys
import os
from pathlib import Path
file = Path(os.getcwd()).resolve()
parent = file.parent
sys.path.append(str(parent))

from workers import common as cmn

In [5]:
sql_scheme=cmn.read_json('../assets/sql_scheme.jsonc')

In [6]:
c=wb.search_countries(".*")

In [7]:
countries = [(g['iso2Code'], g['name'], g['region']['iso2code'],g['region']['value'])
for g in c if g['region']['value'] != 'Aggregates']

In [12]:
cur=pd.read_csv("../assets/currencies.csv")
cur = cur.loc[cur['withdrawal_date'].isna(),:]
# some cleaning
# replace '(abc)' with ', abc'
cur["Entity"]=cur["Entity"].apply(lambda x: re.sub("\\s*\(",", ",x))
cur["Entity"]=cur["Entity"].apply(lambda x: re.sub("\)$","",x))
cur["Entity"]=cur["Entity"].apply(lambda x: re.sub("CAYMAN ISLANDS, THE","CAYMAN ISLANDS",x))
cur["Entity"]=cur["Entity"].apply(lambda x: re.sub("ST.MARTIN, FRENCH PART","ST.MARTIN (FRENCH PART)",x))
cur["Entity"]=cur["Entity"].apply(lambda x: re.sub("ST.MARTIN, FRENCH PART","ST.MARTIN (FRENCH PART)",x))


In [10]:
con=pd.DataFrame(countries, columns=["iso2","country","iso2_region","region"])
con = con.apply(lambda x: x.str.upper())

In [13]:
geo = con.merge(right=cur,how="inner",left_on="country", right_on="Entity")
geo = geo[["iso2","country","iso2_region","region","currency","code","numeric_code"]]
geo['last_upd'] = date.today()

In [2]:
wbdata.get_source()

  id  name
----  --------------------------------------------------------------------
   1  Doing Business
   2  World Development Indicators
   3  Worldwide Governance Indicators
   5  Subnational Malnutrition Database
   6  International Debt Statistics
  11  Africa Development Indicators
  12  Education Statistics
  13  Enterprise Surveys
  14  Gender Statistics
  15  Global Economic Monitor
  16  Health Nutrition and Population Statistics
  18  IDA Results Measurement System
  19  Millennium Development Goals
  20  Quarterly Public Sector Debt
  22  Quarterly External Debt Statistics SDDS
  23  Quarterly External Debt Statistics GDDS
  25  Jobs
  27  Global Economic Prospects
  28  Global Financial Inclusion
  29  The Atlas of Social Protection: Indicators of Resilience and Equity
  30  Exporter Dynamics Database – Indicators at Country-Year Level
  31  Country Policy and Institutional Assessment
  32  Global Financial Development
  33  G20 Financial Inclusion Indicators
  34  Glob

In [43]:
wbdata.get_topic()

  id  value
----  -------------------------------
   1  Agriculture & Rural Development
   2  Aid Effectiveness
   3  Economy & Growth
   4  Education
   5  Energy & Mining
   6  Environment
   7  Financial Sector
   8  Health
   9  Infrastructure
  10  Social Protection & Labor
  11  Poverty
  12  Private Sector
  13  Public Sector
  14  Science & Technology
  15  Social Development
  16  Urban Development
  17  Gender
  18  Millenium development goals
  19  Climate Change
  20  External Debt
  21  Trade

In [84]:
wb.search_indicators(query='stock', topic=7)

id                 name
-----------------  ----------------------------------------------------
CM.MKT.TRAD.CD     Stocks traded, total value (current US$)
CM.MKT.TRAD.GD.ZS  Stocks traded, total value (% of GDP)
CM.MKT.TRNR        Stocks traded, turnover ratio of domestic shares (%)
GFDD.AM.04         Investments financed by equity or stock sales (%)
GFDD.DM.01         Stock market capitalization to GDP (%)
GFDD.DM.02         Stock market total value traded to GDP (%)
GFDD.EM.01         Stock market turnover ratio (%)
GFDD.OM.02         Stock market return (%, year-on-year)
GFDD.SM.01         Stock price volatility
SM.POP.TOTL        International migrant stock, total
SM.POP.TOTL.ZS     International migrant stock (% of population)

In [40]:
wbdata.search_indicators(query='GDP', topic=3)

id                    name
--------------------  ------------------------------------------------------------------------
6.0.GDP_current       GDP (current $)
6.0.GDP_growth        GDP growth (annual %)
6.0.GDP_usd           GDP (constant 2005 $)
6.0.GDPpc_constant    GDP per capita, PPP (constant 2011 international $)
BG.GSR.NFSV.GD.ZS     Trade in services (% of GDP)
BM.KLT.DINV.WD.GD.ZS  Foreign direct investment, net outflows (% of GDP)
BN.CAB.XOKA.GD.ZS     Current account balance (% of GDP)
BN.KLT.PRVT.GD.ZS     Private capital flows, total (% of GDP)
BX.KLT.DINV.WD.GD.ZS  Foreign direct investment, net inflows (% of GDP)
BX.TRF.PWKR.DT.GD.ZS  Personal remittances, received (% of GDP)
GC.DOD.TOTL.GD.ZS     Central government debt, total (% of GDP)
GC.REV.XGRT.GD.ZS     Revenue, excluding grants (% of GDP)
GC.XPN.TOTL.GD.ZS     Expense (% of GDP)
NE.CON.GOVT.ZS        General government final consumption expenditure (% of GDP)
NE.CON.PETC.ZS        Household final consumption exp

In [45]:
wbdata.search_countries('poland')

id    name
----  ------
POL   Poland

In [86]:
[wbdata.get_indicator(['CM.MKT.LCAP.CD']),wbdata.get_indicator('NY.GDP.MKTP.CD'), wbdata.get_indicator('CM.MKT.TRAD.CD')]

[id              name
 --------------  ----------------------------------------------------------------
 CM.MKT.LCAP.CD  Market capitalization of listed domestic companies (current US$),
 id              name
 --------------  -----------------
 NY.GDP.MKTP.CD  GDP (current US$),
 id              name
 --------------  ----------------------------------------
 CM.MKT.TRAD.CD  Stocks traded, total value (current US$)]

In [89]:
# GDP (current US$)
GDP = wbdata.get_data('NY.GDP.MKTP.CD',country='POL', pandas=True)
# Stocks traded, total value (current US$)
stock = wbdata.get_data('CM.MKT.TRAD.CD', country='POL', pandas=True)
# Market capitalization of listed domestic companies (current US$)
cap = wbdata.get_data('CM.MKT.LCAP.CD', country='POL', pandas=True)

In [90]:
stock = stock.dropna().sort_index(ascending=False)
stock_val = stock.iloc[0]
stock_yr = stock.index[0]

GDP = GDP.dropna().sort_index(ascending=False)
gdp_val = GDP.iloc[0]
gdp_yr = GDP.index[0]

cap = cap.dropna().sort_index(ascending=False)
cap_val = cap.iloc[0]
cap_yr = cap.index[0]

In [91]:
[stock_val, stock_yr,gdp_val,gdp_yr, cap_val,cap_yr]

[83438270000.0, '2020', 674048266397.369, '2021', 177508140000.0, '2020']

In [120]:
conti = pd.read_csv('./continent_codes.csv', keep_default_na = False)

'Africa'

In [129]:
g = geocoder.geonames('Teheran', key='stooq_app')
g_id = g.geonames_id
g_det = geocoder.geonames(g_id, method='details', key='stooq_app')

In [130]:
[g_det.continent,
 conti.loc[conti.loc[:, 'Code'] == g_det.continent,'Name'].to_string(index=False),
 g_det.country_code,
 g_det.class_description]


['AS', 'Asia', 'IR', 'city, village,...']